## 148 yyyymmあたりのamount, transaction

In [2]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime

from tqdm import tqdm
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
from sklearn.metrics import mean_squared_error

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

### Data Load

In [4]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_hist = utils.read_df_pkl('../input/hist*0*')
df_new = utils.read_df_pkl('../input/new*0*')

def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

100%|██████████| 3/3 [00:00<00:00, 101.81it/s]

(201917, 6)


SystemExit: 

In [26]:
df = df_new
df.sort_values(by=[key, 'purchase_date'], inplace=True)
df['authorized_flag'] =  df['authorized_flag'].map(lambda x: 0 if x=='N' else 1)
df['category_1'] =  df['category_1'].map(lambda x: 0 if x=='N' else 1)
df['purchase_date'] = pd.to_datetime(df['purchase_date'])
df['purchase_date_shift1'] =  df.groupby(key)['purchase_date'].shift(1)
df['diff_days'] =  (df['purchase_date'] - df['purchase_date_shift1']).dt.days
df = df[[key, 'subsector_id', 'state_id', 'city_id', 'merchant_category_id', 'authorized_flag', 'category_1', 'purchase_amount', 'installments', 'diff_days']]
df_new = df.copy()
del df
gc.collect()

df = df_hist
df.sort_values(by=[key, 'purchase_date'], inplace=True)
df['authorized_flag'] =  df['authorized_flag'].map(lambda x: 0 if x=='N' else 1)
df['category_1'] =  df['category_1'].map(lambda x: 0 if x=='N' else 1)
df['purchase_date'] = pd.to_datetime(df['purchase_date'])
df['purchase_date_shift1'] =  df.groupby(key)['purchase_date'].shift(1)
df['diff_days'] =  (df['purchase_date'] - df['purchase_date_shift1']).dt.days
df = df[[key, 'subsector_id', 'state_id', 'city_id', 'merchant_category_id', 'authorized_flag', 'category_1', 'purchase_amount', 'installments', 'diff_days']]
df_hist = df.copy()
del df
gc.collect()
df_hist.head()

SystemExit: 

In [37]:
auth1 = df_hist[df_hist['authorized_flag']==1]
auth0 = df_hist[df_hist['authorized_flag']==0]
df_train.set_index(key, inplace=True)
df_test.set_index(key, inplace=True)

### カテゴリを横持ちにして、各月の特徴量にする

In [ ]:
cat_list = ['subsector_id', 'state_id']
cat_list = ['merchant_category_id']

# auth0
df = auth0.copy()
fname = 'auth0'
df['record'] = 1

aggs = {}
aggs['category_1'] = ['mean', 'std', 'sum']
aggs['purchase_amount'] = ['mean', 'max', 'min', 'sum']
aggs['diff_days'] = ['mean', 'std']
aggs['installments'] = ['mean', 'max']
# aggs['record'] = ['sum']

for cat in tqdm(cat_list):

    tmp = df.groupby([key, cat]).agg(aggs)
    new_columns = get_new_columns(fname, aggs)
    tmp.columns = new_columns
    
    use_cols = tmp.columns
    tmp_list = []
    tmp2_list = []
    for col in tqdm(use_cols):
        id_list = np.array(tmp.index)
        id_split = np.array_split(id_list, 10)
        for ids in tqdm(id_split):
            tmp2 = tmp.loc[ids, :].unstack()[col]
            for month in tmp2.columns:
                tmp2.rename(columns={month:f"{cat}_{month}_{col}"}, inplace=True)
            tmp2_list.append(tmp2)
        tmp2 = pd.concat(tmp2_list, axis=0)
        tmp_list.append(tmp2)
    
df_auth0 = pd.concat(tmp_list, axis=1)
df_auth0.head()














  0%|          | 0/1 [00:00<?, ?it/s]













  0%|          | 0/11 [00:00<?, ?it/s]














  0%|          | 0/10 [00:00<?, ?it/s]














 10%|█         | 1/10 [00:08<01:13,  8.17s/it]














 20%|██        | 2/10 [00:16<01:06,  8.27s/it]














 30%|███       | 3/10 [00:24<00:57,  8.25s/it]














 40%|████      | 4/10 [00:33<00:49,  8.33s/it]














 50%|█████     | 5/10 [00:41<00:41,  8.25s/it]














 60%|██████    | 6/10 [00:49<00:32,  8.19s/it]














 70%|███████   | 7/10 [00:58<00:24,  8.29s/it]














 80%|████████  | 8/10 [01:06<00:16,  8.20s/it]














 90%|█████████ | 9/10 [01:13<00:08,  8.10s/it]














100%|██████████| 10/10 [01:23<00:00,  8.62s/it]




























  9%|▉         | 1/11 [01:25<14:13, 85.36s/it]














  0%|          | 0/10 [00:00<?, ?it/s]














 10%|█         | 1/10 [00:12<01:48, 12.03s/it]














 20%|██        | 2/10 [00:23<0

In [34]:
cat_list = ['subsector_id', 'state_id']
# cat_list = ['merchant_category_id']

# auth1
df = auth1.copy()
fname = 'auth1'
df['record'] = 1

aggs = {}
aggs['category_1'] = ['mean', 'std', 'sum']
aggs['purchase_amount'] = ['mean', 'max', 'min', 'sum']
aggs['diff_days'] = ['mean', 'std']
aggs['installments'] = ['mean', 'max']
# aggs['record'] = ['sum']

for cat in tqdm(cat_list):

    tmp = df.groupby([key, cat]).agg(aggs)
    new_columns = get_new_columns(fname, aggs)
    tmp.columns = new_columns
    
    use_cols = tmp.columns
    tmp_list = []
    for col in tqdm(use_cols):
        tmp2 = tmp.unstack()[col]
        for month in tmp2.columns:
            tmp2.rename(columns={month:f"{cat}_{month}_{col}"}, inplace=True)
        tmp_list.append(tmp2)
    
df_auth1 = pd.concat(tmp_list, axis=1)
df_auth1.head()







  0%|          | 0/2 [00:00<?, ?it/s]






  0%|          | 0/11 [00:00<?, ?it/s]






  9%|▉         | 1/11 [00:06<01:06,  6.65s/it]






 18%|█▊        | 2/11 [00:13<00:59,  6.65s/it]






 27%|██▋       | 3/11 [00:20<00:53,  6.71s/it]






 36%|███▋      | 4/11 [00:27<00:48,  6.88s/it]






 45%|████▌     | 5/11 [00:34<00:41,  7.00s/it]






 55%|█████▍    | 6/11 [00:42<00:35,  7.09s/it]






 64%|██████▎   | 7/11 [00:49<00:28,  7.15s/it]






 73%|███████▎  | 8/11 [00:56<00:21,  7.21s/it]






 82%|████████▏ | 9/11 [01:04<00:14,  7.27s/it]






 91%|█████████ | 10/11 [01:11<00:07,  7.31s/it]






100%|██████████| 11/11 [01:18<00:00,  7.33s/it]












 50%|█████     | 1/2 [01:29<01:29, 89.04s/it]






  0%|          | 0/11 [00:00<?, ?it/s]






  9%|▉         | 1/11 [00:02<00:26,  2.66s/it]






 18%|█▊        | 2/11 [00:05<00:24,  2.67s/it]






 27%|██▋       | 3/11 [00:08<00:21,  2.69s/it]






 36%|███▋      | 4/11 [00:10<00:18,  2.68s/it]






 45%|

state_id,state_id_-1_auth1_category_1_mean,state_id_1_auth1_category_1_mean,state_id_2_auth1_category_1_mean,state_id_3_auth1_category_1_mean,state_id_4_auth1_category_1_mean,state_id_5_auth1_category_1_mean,state_id_6_auth1_category_1_mean,state_id_7_auth1_category_1_mean,state_id_8_auth1_category_1_mean,state_id_9_auth1_category_1_mean,state_id_10_auth1_category_1_mean,state_id_11_auth1_category_1_mean,state_id_12_auth1_category_1_mean,state_id_13_auth1_category_1_mean,state_id_14_auth1_category_1_mean,state_id_15_auth1_category_1_mean,state_id_16_auth1_category_1_mean,state_id_17_auth1_category_1_mean,state_id_18_auth1_category_1_mean,state_id_19_auth1_category_1_mean,state_id_20_auth1_category_1_mean,state_id_21_auth1_category_1_mean,state_id_22_auth1_category_1_mean,state_id_23_auth1_category_1_mean,state_id_24_auth1_category_1_mean,state_id_-1_auth1_category_1_std,state_id_1_auth1_category_1_std,state_id_2_auth1_category_1_std,state_id_3_auth1_category_1_std,state_id_4_auth1_category_1_std,state_id_5_auth1_category_1_std,state_id_6_auth1_category_1_std,state_id_7_auth1_category_1_std,state_id_8_auth1_category_1_std,state_id_9_auth1_category_1_std,state_id_10_auth1_category_1_std,state_id_11_auth1_category_1_std,state_id_12_auth1_category_1_std,state_id_13_auth1_category_1_std,state_id_14_auth1_category_1_std,state_id_15_auth1_category_1_std,state_id_16_auth1_category_1_std,state_id_17_auth1_category_1_std,state_id_18_auth1_category_1_std,state_id_19_auth1_category_1_std,state_id_20_auth1_category_1_std,state_id_21_auth1_category_1_std,state_id_22_auth1_category_1_std,state_id_23_auth1_category_1_std,state_id_24_auth1_category_1_std,state_id_-1_auth1_category_1_sum,state_id_1_auth1_category_1_sum,state_id_2_auth1_category_1_sum,state_id_3_auth1_category_1_sum,state_id_4_auth1_category_1_sum,state_id_5_auth1_category_1_sum,state_id_6_auth1_category_1_sum,state_id_7_auth1_category_1_sum,state_id_8_auth1_category_1_sum,state_id_9_auth1_category_1_sum,state_id_10_auth1_category_1_sum,state_id_11_auth1_category_1_sum,state_id_12_auth1_category_1_sum,state_id_13_auth1_category_1_sum,state_id_14_auth1_category_1_sum,state_id_15_auth1_category_1_sum,state_id_16_auth1_category_1_sum,state_id_17_auth1_category_1_sum,state_id_18_auth1_category_1_sum,state_id_19_auth1_category_1_sum,state_id_20_auth1_category_1_sum,state_id_21_auth1_category_1_sum,state_id_22_auth1_category_1_sum,state_id_23_auth1_category_1_sum,state_id_24_auth1_category_1_sum,state_id_-1_auth1_purchase_amount_mean,state_id_1_auth1_purchase_amount_mean,state_id_2_auth1_purchase_amount_mean,state_id_3_auth1_purchase_amount_mean,state_id_4_auth1_purchase_amount_mean,state_id_5_auth1_purchase_amount_mean,state_id_6_auth1_purchase_amount_mean,state_id_7_auth1_purchase_amount_mean,state_id_8_auth1_purchase_amount_mean,state_id_9_auth1_purchase_amount_mean,state_id_10_auth1_purchase_amount_mean,state_id_11_auth1_purchase_amount_mean,state_id_12_auth1_purchase_amount_mean,state_id_13_auth1_purchase_amount_mean,state_id_14_auth1_purchase_amount_mean,state_id_15_auth1_purchase_amount_mean,state_id_16_auth1_purchase_amount_mean,state_id_17_auth1_purchase_amount_mean,state_id_18_auth1_purchase_amount_mean,state_id_19_auth1_purchase_amount_mean,state_id_20_auth1_purchase_amount_mean,state_id_21_auth1_purchase_amount_mean,state_id_22_auth1_purchase_amount_mean,state_id_23_auth1_purchase_amount_mean,state_id_24_auth1_purchase_amount_mean,...,state_id_-1_auth1_diff_days_mean,state_id_1_auth1_diff_days_mean,state_id_2_auth1_diff_days_mean,state_id_3_auth1_diff_days_mean,state_id_4_auth1_diff_days_mean,state_id_5_auth1_diff_days_mean,state_id_6_auth1_diff_days_mean,state_id_7_auth1_diff_days_mean,state_id_8_auth1_diff_days_mean,state_id_9_auth1_diff_days_mean,state_id_10_auth1_diff_days_mean,state_id_11_auth1_diff_days_mean,state_id_12_auth1_diff_days_mean,state_id_13_auth1_diff_days_mean,state_id_14_auth1_diff_days_mean,state_id_15_auth1_diff_days_mean,state_id_16_auth1_diff_days_m

In [35]:
cat_list = ['subsector_id', 'state_id']
# cat_list = ['merchant_category_id']

# new
df = df_new.copy()
fname = 'new'
df['record'] = 1

aggs = {}
aggs['category_1'] = ['mean', 'std', 'sum']
aggs['purchase_amount'] = ['mean', 'max', 'min', 'sum']
aggs['diff_days'] = ['mean', 'std']
aggs['installments'] = ['mean', 'max']
# aggs['record'] = ['sum']

for cat in tqdm(cat_list):

    tmp = df.groupby([key, cat]).agg(aggs)
    new_columns = get_new_columns(fname, aggs)
    tmp.columns = new_columns
    
    use_cols = tmp.columns
    tmp_list = []
    for col in tqdm(use_cols):
        tmp2 = tmp.unstack()[col]
        for month in tmp2.columns:
            tmp2.rename(columns={month:f"{cat}_{month}_{col}"}, inplace=True)
        tmp_list.append(tmp2)
    
df_new_feat = pd.concat(tmp_list, axis=1)
df_new_feat.head()







  0%|          | 0/2 [00:00<?, ?it/s]






  0%|          | 0/11 [00:00<?, ?it/s]






  9%|▉         | 1/11 [00:04<00:41,  4.14s/it]






 18%|█▊        | 2/11 [00:08<00:37,  4.15s/it]






 27%|██▋       | 3/11 [00:12<00:33,  4.15s/it]






 36%|███▋      | 4/11 [00:16<00:28,  4.14s/it]






 45%|████▌     | 5/11 [00:20<00:24,  4.14s/it]






 55%|█████▍    | 6/11 [00:25<00:21,  4.22s/it]






 64%|██████▎   | 7/11 [00:29<00:17,  4.40s/it]






 73%|███████▎  | 8/11 [00:34<00:13,  4.54s/it]






 82%|████████▏ | 9/11 [00:39<00:09,  4.64s/it]






 91%|█████████ | 10/11 [00:44<00:04,  4.71s/it]






100%|██████████| 11/11 [00:49<00:00,  4.75s/it]












 50%|█████     | 1/2 [00:50<00:50, 50.94s/it]






  0%|          | 0/11 [00:00<?, ?it/s]






  9%|▉         | 1/11 [00:02<00:21,  2.16s/it]






 18%|█▊        | 2/11 [00:04<00:19,  2.16s/it]






 27%|██▋       | 3/11 [00:06<00:17,  2.16s/it]






 36%|███▋      | 4/11 [00:08<00:15,  2.16s/it]






 45%|

state_id,state_id_-1_new_category_1_mean,state_id_1_new_category_1_mean,state_id_2_new_category_1_mean,state_id_3_new_category_1_mean,state_id_4_new_category_1_mean,state_id_5_new_category_1_mean,state_id_6_new_category_1_mean,state_id_7_new_category_1_mean,state_id_8_new_category_1_mean,state_id_9_new_category_1_mean,state_id_10_new_category_1_mean,state_id_11_new_category_1_mean,state_id_12_new_category_1_mean,state_id_13_new_category_1_mean,state_id_14_new_category_1_mean,state_id_15_new_category_1_mean,state_id_16_new_category_1_mean,state_id_17_new_category_1_mean,state_id_18_new_category_1_mean,state_id_19_new_category_1_mean,state_id_20_new_category_1_mean,state_id_21_new_category_1_mean,state_id_22_new_category_1_mean,state_id_23_new_category_1_mean,state_id_24_new_category_1_mean,state_id_-1_new_category_1_std,state_id_1_new_category_1_std,state_id_2_new_category_1_std,state_id_3_new_category_1_std,state_id_4_new_category_1_std,state_id_5_new_category_1_std,state_id_6_new_category_1_std,state_id_7_new_category_1_std,state_id_8_new_category_1_std,state_id_9_new_category_1_std,state_id_10_new_category_1_std,state_id_11_new_category_1_std,state_id_12_new_category_1_std,state_id_13_new_category_1_std,state_id_14_new_category_1_std,state_id_15_new_category_1_std,state_id_16_new_category_1_std,state_id_17_new_category_1_std,state_id_18_new_category_1_std,state_id_19_new_category_1_std,state_id_20_new_category_1_std,state_id_21_new_category_1_std,state_id_22_new_category_1_std,state_id_23_new_category_1_std,state_id_24_new_category_1_std,state_id_-1_new_category_1_sum,state_id_1_new_category_1_sum,state_id_2_new_category_1_sum,state_id_3_new_category_1_sum,state_id_4_new_category_1_sum,state_id_5_new_category_1_sum,state_id_6_new_category_1_sum,state_id_7_new_category_1_sum,state_id_8_new_category_1_sum,state_id_9_new_category_1_sum,state_id_10_new_category_1_sum,state_id_11_new_category_1_sum,state_id_12_new_category_1_sum,state_id_13_new_category_1_sum,state_id_14_new_category_1_sum,state_id_15_new_category_1_sum,state_id_16_new_category_1_sum,state_id_17_new_category_1_sum,state_id_18_new_category_1_sum,state_id_19_new_category_1_sum,state_id_20_new_category_1_sum,state_id_21_new_category_1_sum,state_id_22_new_category_1_sum,state_id_23_new_category_1_sum,state_id_24_new_category_1_sum,state_id_-1_new_purchase_amount_mean,state_id_1_new_purchase_amount_mean,state_id_2_new_purchase_amount_mean,state_id_3_new_purchase_amount_mean,state_id_4_new_purchase_amount_mean,state_id_5_new_purchase_amount_mean,state_id_6_new_purchase_amount_mean,state_id_7_new_purchase_amount_mean,state_id_8_new_purchase_amount_mean,state_id_9_new_purchase_amount_mean,state_id_10_new_purchase_amount_mean,state_id_11_new_purchase_amount_mean,state_id_12_new_purchase_amount_mean,state_id_13_new_purchase_amount_mean,state_id_14_new_purchase_amount_mean,state_id_15_new_purchase_amount_mean,state_id_16_new_purchase_amount_mean,state_id_17_new_purchase_amount_mean,state_id_18_new_purchase_amount_mean,state_id_19_new_purchase_amount_mean,state_id_20_new_purchase_amount_mean,state_id_21_new_purchase_amount_mean,state_id_22_new_purchase_amount_mean,state_id_23_new_purchase_amount_mean,state_id_24_new_purchase_amount_mean,...,state_id_-1_new_diff_days_mean,state_id_1_new_diff_days_mean,state_id_2_new_diff_days_mean,state_id_3_new_diff_days_mean,state_id_4_new_diff_days_mean,state_id_5_new_diff_days_mean,state_id_6_new_diff_days_mean,state_id_7_new_diff_days_mean,state_id_8_new_diff_days_mean,state_id_9_new_diff_days_mean,state_id_10_new_diff_days_mean,state_id_11_new_diff_days_mean,state_id_12_new_diff_days_mean,state_id_13_new_diff_days_mean,state_id_14_new_diff_days_mean,state_id_15_new_diff_days_mean,state_id_16_new_diff_days_mean,state_id_17_new_diff_days_mean,state_id_18_new_diff_days_mean,state_id_19_new_diff_days_mean,state_id_20_new_diff_days_mean,state_id_21_new_diff_days_mean,state_id_22_new_diff_days_mean,state_id_23_new_diff_days_mean,state_id_24_n

In [39]:
for df in [df_auth1, df_auth0, df_new_feat]:
# for df in [df_hist_feat]:
    df_train = df_train.join(df)
    df_test = df_test.join(df)

In [40]:
df_train.head()

,first_active_month,feature_1,feature_2,feature_3,target,state_id_-1_auth1_category_1_mean,state_id_1_auth1_category_1_mean,state_id_2_auth1_category_1_mean,state_id_3_auth1_category_1_mean,state_id_4_auth1_category_1_mean,state_id_5_auth1_category_1_mean,state_id_6_auth1_category_1_mean,state_id_7_auth1_category_1_mean,state_id_8_auth1_category_1_mean,state_id_9_auth1_category_1_mean,state_id_10_auth1_category_1_mean,state_id_11_auth1_category_1_mean,state_id_12_auth1_category_1_mean,state_id_13_auth1_category_1_mean,state_id_14_auth1_category_1_mean,state_id_15_auth1_category_1_mean,state_id_16_auth1_category_1_mean,state_id_17_auth1_category_1_mean,state_id_18_auth1_category_1_mean,state_id_19_auth1_category_1_mean,state_id_20_auth1_category_1_mean,state_id_21_auth1_category_1_mean,state_id_22_auth1_category_1_mean,state_id_23_auth1_category_1_mean,state_id_24_auth1_category_1_mean,state_id_-1_auth1_category_1_std,state_id_1_auth1_category_1_std,state_id_2_auth1_category_1_std,state_id_3_auth1_category_1_std,state_id_4_auth1_category_1_std,state_id_5_auth1_category_1_std,state_id_6_auth1_category_1_std,state_id_7_auth1_category_1_std,state_id_8_auth1_category_1_std,state_id_9_auth1_category_1_std,state_id_10_auth1_category_1_std,state_id_11_auth1_category_1_std,state_id_12_auth1_category_1_std,state_id_13_auth1_category_1_std,state_id_14_auth1_category_1_std,state_id_15_auth1_category_1_std,state_id_16_auth1_category_1_std,state_id_17_auth1_category_1_std,state_id_18_auth1_category_1_std,state_id_19_auth1_category_1_std,state_id_20_auth1_category_1_std,state_id_21_auth1_category_1_std,state_id_22_auth1_category_1_std,state_id_23_auth1_category_1_std,state_id_24_auth1_category_1_std,state_id_-1_auth1_category_1_sum,state_id_1_auth1_category_1_sum,state_id_2_auth1_category_1_sum,state_id_3_auth1_category_1_sum,state_id_4_auth1_category_1_sum,state_id_5_auth1_category_1_sum,state_id_6_auth1_category_1_sum,state_id_7_auth1_category_1_sum,state_id_8_auth1_category_1_sum,state_id_9_auth1_category_1_sum,state_id_10_auth1_category_1_sum,state_id_11_auth1_category_1_sum,state_id_12_auth1_category_1_sum,state_id_13_auth1_category_1_sum,state_id_14_auth1_category_1_sum,state_id_15_auth1_category_1_sum,state_id_16_auth1_category_1_sum,state_id_17_auth1_category_1_sum,state_id_18_auth1_category_1_sum,state_id_19_auth1_category_1_sum,state_id_20_auth1_category_1_sum,state_id_21_auth1_category_1_sum,state_id_22_auth1_category_1_sum,state_id_23_auth1_category_1_sum,state_id_24_auth1_category_1_sum,state_id_-1_auth1_purchase_amount_mean,state_id_1_auth1_purchase_amount_mean,state_id_2_auth1_purchase_amount_mean,state_id_3_auth1_purchase_amount_mean,state_id_4_auth1_purchase_amount_mean,state_id_5_auth1_purchase_amount_mean,state_id_6_auth1_purchase_amount_mean,state_id_7_auth1_purchase_amount_mean,state_id_8_auth1_purchase_amount_mean,state_id_9_auth1_purchase_amount_mean,state_id_10_auth1_purchase_amount_mean,state_id_11_auth1_purchase_amount_mean,state_id_12_auth1_purchase_amount_mean,state_id_13_auth1_purchase_amount_mean,state_id_14_auth1_purchase_amount_mean,state_id_15_auth1_purchase_amount_mean,state_id_16_auth1_purchase_amount_mean,state_id_17_auth1_purchase_amount_mean,state_id_18_auth1_purchase_amount_mean,state_id_19_auth1_purchase_amount_mean,...,state_id_-1_new_diff_days_mean,state_id_1_new_diff_days_mean,state_id_2_new_diff_days_mean,state_id_3_new_diff_days_mean,state_id_4_new_diff_days_mean,state_id_5_new_diff_days_mean,state_id_6_new_diff_days_mean,state_id_7_new_diff_days_mean,state_id_8_new_diff_days_mean,state_id_9_new_diff_days_mean,state_id_10_new_diff_days_mean,state_id_11_new_diff_days_mean,state_id_12_new_diff_days_mean,state_id_13_new_diff_days_mean,state_id_14_new_diff_days_mean,state_id_15_new_diff_days_mean,state_id_16_new_diff_days_mean,state_id_17_new_diff_days_mean,state_id_18_new_diff_days_mean,state_id_19_new_diff_days_mean,state_id_20_new_diff_days_mean,state_id_21_new_diff_days_mean,state_id_22_new_diff_d

### make ratio feature

In [41]:
fname = '148_sss'
ignore_features = ['first_active_month', 'card_id', target]

for col in df_train.columns:
    if col in ignore_features: continue
    if not(col.count('_id_')):continue
    length = len(df_train[col].dropna())
    if length/len(df_train)<0.02:
        print(col)
        continue
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_train_{col}@', obj=df_train[col].values)
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_test_{col}@', obj=df_test[col].values)

state_id_6_auth1_category_1_mean
state_id_8_auth1_category_1_mean
state_id_10_auth1_category_1_mean
state_id_14_auth1_category_1_mean
state_id_17_auth1_category_1_mean
state_id_23_auth1_category_1_mean
state_id_6_auth1_category_1_std
state_id_8_auth1_category_1_std
state_id_10_auth1_category_1_std
state_id_14_auth1_category_1_std
state_id_17_auth1_category_1_std
state_id_23_auth1_category_1_std
state_id_24_auth1_category_1_std
state_id_6_auth1_category_1_sum
state_id_8_auth1_category_1_sum
state_id_10_auth1_category_1_sum
state_id_14_auth1_category_1_sum
state_id_17_auth1_category_1_sum
state_id_23_auth1_category_1_sum
state_id_6_auth1_purchase_amount_mean
state_id_8_auth1_purchase_amount_mean
state_id_10_auth1_purchase_amount_mean
state_id_14_auth1_purchase_amount_mean
state_id_17_auth1_purchase_amount_mean
state_id_23_auth1_purchase_amount_mean
state_id_6_auth1_purchase_amount_max
state_id_8_auth1_purchase_amount_max
state_id_10_auth1_purchase_amount_max
state_id_14_auth1_purchase_am

In [37]:
import glob

p_list = glob.glob('../features/1_first_valid/*.gz')

for p in p_list:
    if p.count('test'):continue
    tmp = utils.read_pkl_gzip(p)
    if len(tmp)<200000:
        print(len(tmp))